# 2: Regular Expressions, Text Normalization, Edit Distance

Solution by: Peter Chang (GItHub: @peterchang0414)

*Last edited: 2021.12.16*

**2.1**: Write regular expressions for the following languages.


1.   the set of all alphabetic strings;
2.   the set of all lower case alphabetic strings ending in a *b*
3.   the set of all strings from the alphabet *a, b* such that each *a* is immediately preceded by and immediately followed by a *b*;



In [1]:
import re

strings = ["a_b0c", "Db", "AbcdE", "bbabbbabbb", "abab", "123", "1b", "bb"]

# We interpret "string" as an entire word in 'strings', a list of words
print("1. The set of all alphabetic strings:")
for s in strings:
  match_1 = re.search(r'^[a-zA-Z]+$', s)
  if match_1:
    print(' ', match_1.group())
print('\n')

print("2. The set of all lower-case alphabetic strings ending in a 'b':")
for s in strings:
  match_2 = re.search(r'^[a-z]*b$', s)
  if match_2:
    print(' ', match_2.group())
print('\n')

print("3. The set of all strings from the alphabet [a,b] such that\n",\
      "each a is immediately preceded by and immediately followed by a b:")
for s in strings:
  match_3 = re.search(r'^b+(?:ab+)*b*$', s)
  if match_3:
    print(' ', match_3.group())

1. The set of all alphabetic strings:
  Db
  AbcdE
  bbabbbabbb
  abab
  bb


2. The set of all lower-case alphabetic strings ending in a 'b':
  bbabbbabbb
  abab
  bb


3. The set of all strings from the alphabet [a,b] such that
 each a is immediately preceded by and immediately followed by a b:
  bbabbbabbb
  bb


**2.2**: Write regular expressions for the following languages. By "word", we mean an alphabetic string separated from other words by whitespace, any relevant punctuation, line breaks, and so forth.


1.   the set of all strings with two consecutive repeated words (e.g., "Humbert Humbert" and "the the" but not "the bug" or "the big bug");
2.   all strings that start at the beginning of the line with an integer and that end at the end of the line with a word;
3.   all strings that have both the word *grotto* and the word *raven* in them (but not, e.g., words llike *grottos* that merely *contain* the word *grotto*);
4.   write a pattern that places the first word of an English sentence in a register. Deal with punctuation.



In [2]:
import re

strings = ["Humbert Humbert", "thethe", "the big bug", "the bug bug",\
         "1,one", "2a two", "3:three", "4four", "5 4five", "z0 zero",\
         "grotto raven", "grottos raven", "grottoraven", "h raven grotto h"]

print("1. The set of all strings with two consecutive repeated words:")
for s in strings:
  match_1 = re.search(r'(?:^|^.+\W)([a-zA-Z]+)\W+\1(?:$|\W.+$)', s)
  if match_1:
    print(' ', match_1.group())
print('\n')

print("2. All strings that start at the beginning of the line with an",\
      "integer and that end at the end of the line with a word:")
for s in strings:
  match_2 = re.search(r'^\d.*\W[a-zA-Z]+$', s)
  if match_2:
    print(' ', match_2.group())
print('\n')

print("3. All strings that have both the word grotto and the word raven",\
      "in them:")
for s in strings:
  match_3 = re.search(r'(^|^.+\W)(grotto\W.*raven|raven\W.*grotto)($|\W.+$)', s)
  if match_3:
    print(' ', match_3.group())
print('\n')

print("4. Write a pattern that places the first word of an English sentence",\
      "in a register. Deal with punctuation.")
sentence = "!!Hello, world: this is a regular expression!"
match_4 = re.search(r'^\W*([a-zA-Z]+)(?:$|\W.+$)', sentence)
if match_4 and match_4.group(1):
  # Print the first captured group in the register
  print(' The first word of sentence: "', sentence, '" is: "',\
        match_4.group(1), '"', sep="")

1. The set of all strings with two consecutive repeated words:
  Humbert Humbert
  the bug bug


2. All strings that start at the beginning of the line with an integer and that end at the end of the line with a word:
  1,one
  2a two
  3:three


3. All strings that have both the word grotto and the word raven in them:
  grotto raven
  h raven grotto h


4. Write a pattern that places the first word of an English sentence in a register. Deal with punctuation.
 The first word of sentence: "!!Hello, world: this is a regular expression!" is: "Hello"


**2.3**: Implement an ELIZA-like program, using substitutions such as those described on page 11. You might want to choose a different domain than a Rogerian psychologist, although keep in mind that you would need a domain in which your program can legitimately engage in a lot of simple repetition.

In [80]:
import re, random

# ELIZA
t_b = "eliza: "
user_str = input(t_b + "WHAT'S ON YOUR MIND? (input 'quit' to quit)\n")
while user_str != 'quit':
  response = re.sub(r'.*I\'M (DEPRESSED|SAD|WORRIED).*',\
                    r'I AM SORRY TO HEAR YOU ARE \1', user_str.upper())
  if response == user_str.upper():
    response = re.sub(r'.*I AM (DEPRESSED|SAD|HAPPY|WORRIED).*', \
                      r'WHY DO YOU THINK YOU ARE \1', response)
  if response == user_str.upper():
    response = re.sub(r'.* ALL .*', \
                      r'IN WHAT WAY', response)
  if response == user_str.upper():
    response = re.sub(r'.* ALWAYS .*', \
                      r'CAN YOU THINK OF A SPECIFIC EXAMPLE', response)
  if response == user_str.upper():
    default = ['please, tell me more', 'i understand', 
               'recognizing that is good', 'thank you for telling me']
    default = [st.upper() for st in default]
    response = random.choice(default)
  user_str = input(t_b + response + '\n')

eliza: WHAT'S ON YOUR MIND? (input 'quit' to quit)
Men are all alike
eliza: IN WHAT WAY
They're always bugging us about something or other.
eliza: CAN YOU THINK OF A SPECIFIC EXAMPLE
Well, my boyfriend made me come here.
eliza: PLEASE, TELL ME MORE
He says I'm depressed much of the time.
eliza: I AM SORRY TO HEAR YOU ARE DEPRESSED
quit


**2.4**: Compute the edit distance (using insertion cost 1, deletion cost 1, substitution cost 1) of "leda" to "deal". Show your work (using the edit distance grid).

In [52]:
import pandas as pd

def min_edit_distance(source, target, ins_cost, del_cost, sub_cost):
  n, m = len(source), len(target)
  # initialization
  dp = [[0 for _ in range(m+1)] for _ in range(n+1)]
  for i in range(1, n+1):
    dp[i][0] = dp[i-1][0] + del_cost
  for j in range(1, m+1):
    dp[0][j] = dp[0][j-1] + ins_cost
  # recurrence relation
  for i in range(1, n+1):
    for j in range(1, m+1):
      sub_cost_curr = 0
      if source[i-1] != target[j-1]:
        sub_cost_curr = sub_cost
      dp[i][j] = min(dp[i-1][j]+del_cost, dp[i-1][j-1]+sub_cost_curr, \
                     dp[i][j-1]+ins_cost)
  return dp, dp[n][m]

source = input("Enter source string: ")
target = input("Enter target string: ")
ins_cost = int(input("Enter insertion cost: "))
del_cost = int(input("Enter deletion cost: "))
sub_cost = int(input("Enter substitution cost: "))

dp, distance = min_edit_distance(source, target, ins_cost, del_cost, sub_cost)

print('\n' + "="*30 + '\n')

print("The edit distance between '" + source +"' and '" + target \
      + "' is: " + str(distance) + '\n')
print("The edit distance grid is as follows:")
print(pd.DataFrame(dp, index=list('#'+source), columns=list('#'+target)))

Enter source string: intention
Enter target string: execution
Enter insertion cost: 1
Enter deletion cost: 1
Enter substitution cost: 1


The edit distance between 'intention' and 'execution' is: 5

The edit distance grid is as follows:
   #  e  x  e  c  u  t  i  o  n
#  0  1  2  3  4  5  6  7  8  9
i  1  1  2  3  4  5  6  6  7  8
n  2  2  2  3  4  5  6  7  7  7
t  3  3  3  3  4  5  5  6  7  8
e  4  3  4  3  4  5  6  6  7  8
n  5  4  4  4  4  5  6  7  7  7
t  6  5  5  5  5  5  5  6  7  8
i  7  6  6  6  6  6  6  5  6  7
o  8  7  7  7  7  7  7  6  5  6
n  9  8  8  8  8  8  8  7  6  5


**2.5**: Figure out whether *drive* is closer to *brief* or to *divers* and what the edit distance is to each. You may use any version of *distance* that you like.

In [51]:
drive, brief, divers = 'drive', 'brief', 'divers'
costs = [1, 1, 1]
dp1, dist1 = min_edit_distance(drive, brief, *costs)
dp2, dist2 = min_edit_distance(drive, divers, *costs)
print("The edit distance between '"+drive+"' and '"+brief+"' is "+str(dist1))
print("The edit distance between '"+drive+"' and '"+divers+"' is "+str(dist2))
if dist1 == dist2:
  print("Thus, the edit distances are the same.")
elif dist1 < dist2:
  print("Thus, '"+drive+"' is closer to '"+brief+"'.")
else:
  print("Thus, '"+drive+"' is closer to '"+divers+"'.")

The edit distance between 'drive' and 'brief' is 3
The edit distance between 'drive' and 'divers' is 3
Thus, the edit distances are the same.


**2.6**: Now implement a minimum edit distance algorithm and use your hand-computed results to check your code.

**2.7**: Augment the minimum edit distance algorithm to output an alignment; you will need to store pointers and add a stage to compute the backtrace.

In [78]:
import pandas as pd

def aug_min_edit_distance(source, target, ins_cost, del_cost, sub_cost):
  n, m = len(source), len(target)
  # initialization
  dp = [[0 for _ in range(m+1)] for _ in range(n+1)]
  backpointers = [["" for _ in range(m+1)] for _ in range(n+1)]
  for i in range(1, n+1):
    dp[i][0] = dp[i-1][0] + del_cost
    backpointers[i][0] = "del"
  for j in range(1, m+1):
    dp[0][j] = dp[0][j-1] + ins_cost
    backpointers[0][j] = "ins"
  # recurrence relation
  for i in range(1, n+1):
    for j in range(1, m+1):
      ic, dc, sc = dp[i][j-1]+ins_cost, dp[i-1][j]+del_cost, dp[i-1][j-1]
      if source[i-1] != target[j-1]:
        sc += sub_cost
      if sc == min(ic, dc, sc):
        backpointers[i][j] = "sub"
      elif ic == min(ic, dc, sc):
        backpointers[i][j] = "ins"
      else:
        backpointers[i][j] = "del"
      dp[i][j] = min(ic, dc, sc)
  return backpointers, dp, dp[n][m]

def produce_alignment(source, target, ins_cost, del_cost, sub_cost):
  bp, _, _ = aug_min_edit_distance(source, target, ins_cost, del_cost, sub_cost)
  source, target = source.upper(), target.upper()
  source_aligned, target_aligned = "", ""
  i, j = len(bp)-1, len(bp[0])-1
  while i or j:
    if bp[i][j] == "sub":
      i, j = i-1, j-1
      source_aligned = source[i] + source_aligned
      target_aligned = target[j] + target_aligned
    elif bp[i][j] == "ins":
      j -= 1
      source_aligned = '*'+source_aligned
      target_aligned = target[j] + target_aligned
    else:
      i -= 1
      source_aligned = source[i] + source_aligned
      target_aligned = '*'+target_aligned
  return source_aligned, target_aligned

source = input("Enter source string: ")
target = input("Enter target string: ")
ic = int(input("Enter insertion cost: "))
dc = int(input("Enter deletion cost: "))
sc = int(input("Enter substitution cost: "))

s_aligned, t_aligned = produce_alignment(source, target, ic, dc, sc)

print('\n'+"="*30+'\n')
print("The alignment is as follows:\n")
print(' '*8+s_aligned)
print(' '*8+t_aligned)

Enter source string: Hello, I am Peter Chang
Enter target string: Hello, I am Paul Thomas Anderson
Enter insertion cost: 1
Enter deletion cost: 1
Enter substitution cost: 2


The alignment is as follows:

        HELLO, I AM PETER CH**A***N*****G
        HELLO, I AM P*AUL THOMAS ANDERSON
